In [26]:
import pandas as pd
import sklearn
import numpy as np

In [9]:
from sklearn.ensemble import IsolationForest

## Method Testing

In [16]:
sales = pd.read_csv('sales_mod.csv')

/tmp/ipykernel_1086328/1754691465.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  sales = pd.read_csv('sales_mod.csv')


In [ ]:
sales.drop(columns=['Unnamed: 0'], inplace=True)

In [20]:
sales.set_index('pin', inplace=True)

In [18]:
sales.columns

Index(['township_code', 'class', 'pin', 'year', 'sale_date', 'sale_price',
       'sale_price_log10', 'sale_key', 'doc_no', 'deed_type', 'seller_name',
       'is_multisale', 'num_parcels_sale', 'buyer_name', 'sale_type',
       'sale_filter_lower_limit', 'sale_filter_upper_limit',
       'sale_filter_count', 'property_advertised',
       'is_installment_contract_fulfilled',
       'is_sale_between_related_individuals_or_corporate_affiliates',
       'is_transfer_of_less_than_100_percent_interest',
       'is_court_ordered_sale', 'is_sale_in_lieu_of_foreclosure',
       'is_condemnation', 'is_short_sale', 'is_bank_reo_real_estate_owned',
       'is_auction_sale', 'is_seller_buyer_a_relocation_company',
       'is_seller_buyer_a_financial_institution_or_government_agency',
       'is_buyer_a_real_estate_investment_trust', 'is_buyer_a_pension_fund',
       'is_buyer_an_adjacent_property_owner',
       'is_buyer_exercising_an_option_to_purchase',
       'is_simultaneous_trade_of_property'

In [19]:
feed_data = sales[['sale_price_log10', 'price_per_sqft_log10', 'pct', 'days_since_last_transaction', 'counts', 'town_class_movement', ]]

In [22]:
feed_data.dropna(inplace=True)

/tmp/ipykernel_1086328/3240171540.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feed_data.dropna(inplace=True)


In [23]:
isof = IsolationForest()
prediction = isof.fit_predict(feed_data)

In [28]:
# -1 is outlier and 1 is normal
# Looking at human readable CSV, this looks to be about similar to (2,2) in terms of outlier detections just from raw numbers 
np.unique(prediction, return_counts=True)

(array([-1,  1]), array([ 5013, 21844]))